In [1]:
import pandas as pd
import numpy as np
import os
import dask.dataframe as dd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
items = dd.read_csv('data/items.csv')
holiday = dd.read_csv('data/holidays_events.csv')
oil = pd.read_csv('data/oil.csv')
stores = dd.read_csv('data/stores.csv')
transactions = dd.read_csv('data/transactions.csv')
train_set = pd.read_csv('data/train.csv')
# test_set = pd.read_csv('data/test.csv')

/Users/lkalina/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
items = pd.read_csv('data/items.csv')

In [4]:
train_set = pd.read_csv('data/train.csv')

In [5]:
# Wednesday to Wednesday (2 weeks)
train_set.date.unique()

array(['2013-01-01', '2013-01-02', '2013-01-03', ..., '2017-08-13',
       '2017-08-14', '2017-08-15'], dtype=object)

In [6]:
train_set.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [7]:
train_set['date'] = pd.to_datetime(train_set['date'])

In [8]:
train_set[(train_set.date > '2016-08-15')].head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
88138659,88138659,2016-08-16,1,105574,3.0,False
88138660,88138660,2016-08-16,1,105575,13.0,False
88138661,88138661,2016-08-16,1,105737,1.0,False
88138662,88138662,2016-08-16,1,105857,8.0,False
88138663,88138663,2016-08-16,1,106716,1.0,False


# Run rolling means for test

In [2]:
import pandas as pd
from datetime import timedelta

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}

train = pd.read_csv('data/train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 86672217) #Skip dates before 2016-08-01
                    )

train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek

# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
)

del u_dates, u_stores, u_items

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
train.reset_index(inplace=True) # reset index and restoring unique columns  
lastdate = train.iloc[train.shape[0]-1].date

#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')
ma_dw.reset_index(inplace=True)
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')
ma_wk.reset_index(inplace=True)

#Moving Averages
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')
for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

del tmp,tmpg,train

ma_is['mais']=ma_is.median(axis=1)
ma_is.reset_index(inplace=True)

#Load test
test = pd.read_csv('data/test.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

del ma_is, ma_wk, ma_dw

#Forecasting Test
test['unit_sales'] = test.mais 
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values 

#50% more for promotion items
test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

#test.to_csv('ma8dwof.csv.gz', index=False, float_format='%.3f', compression='gzip')

In [12]:
#test.to_csv('ma8dwof.csv.gz', index=False, float_format='%.3f', compression='gzip')

In [9]:
test.head()

,id,date,store_nbr,item_nbr,onpromotion,dow,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais,mawk,madw,unit_sales
0,125497040,2017-08-16,1,96995,False,2,0.056869,0.154255,0.172356,0.295202,0.334438,0.099021,0.000000,0.000000,0.126638,0.844024,0.693147,0.109601
1,125497041,2017-08-16,1,99197,False,2,0.206482,0.470665,0.397257,0.280721,0.206455,0.156945,0.000000,0.000000,0.206469,1.010969,0.977616,0.220984
2,125497042,2017-08-16,1,103501,False,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000
3,125497043,2017-08-16,1,103520,False,2,0.760955,0.815118,0.773160,0.726315,0.573577,0.495105,0.231049,0.000000,0.649946,1.114774,1.175135,0.984047
4,125497044,2017-08-16,1,103665,False,2,1.014494,1.038379,0.996363,1.026335,1.031388,0.980990,0.462098,0.693147,1.005429,1.320085,1.282500,1.655950


# Run Prophet

In [9]:
data = pd.read_csv('data/modified/one_store.csv',index_col=0)

/Users/lkalina/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
data.family.unique()

array(['BREAD/BAKERY', 'GROCERY I', 'DELI', 'CLEANING', 'PERSONAL CARE',
       'BEVERAGES', 'DAIRY', 'EGGS', 'POULTRY', 'LINGERIE',
       'PREPARED FOODS', 'GROCERY II', 'FROZEN FOODS', 'MEATS',
       'LIQUOR,WINE,BEER', 'LAWN AND GARDEN', 'BEAUTY', 'AUTOMOTIVE',
       'SEAFOOD', 'HARDWARE', 'HOME APPLIANCES', 'PRODUCE',
       'HOME AND KITCHEN II', 'HOME AND KITCHEN I', 'MAGAZINES',
       'HOME CARE', 'PET SUPPLIES', 'PLAYERS AND ELECTRONICS',
       'CELEBRATION', 'SCHOOL AND OFFICE SUPPLIES', 'BABY CARE', 'BOOKS'], dtype=object)

In [11]:
data['y'] = np.log(data['unit_sales'])
data = data[['date','y']].rename(columns={'date':'ds'})

/Users/lkalina/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in log
  if __name__ == '__main__':


In [13]:
import pandas as pd
import numpy as np
from fbprophet import Prophet

In [14]:
items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [15]:
#df[df.onpromotion].unit_sales.max()
#df = df[['date','y']].rename(columns={'date':'ds'})
train_set = train_set.merge(items,on='item_nbr',how='left')

In [16]:
#train_set = pd.read_csv('data/train.csv')
train_set['y'] = np.log(train_set['unit_sales'])
train_set = train_set[['date','y','family']].rename(columns={'date':'ds'})

/Users/lkalina/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app


In [17]:
cats = train_set_set.family.unique()

## Running Predictions for Test

In [18]:
# pred = {}

# for cat in cats:
#     df = train_set[train_set.family == cat]
#     df = df[['ds','y']].groupby('ds').mean()
#     df['ds'] = df.index
#     # Fit the category with Prophet
#     m = Prophet()
#     m.fit(df);
#     future = m.make_future_dataframe(periods=16,freq='D')
#     pred[cat] = m.predict(future)

In [19]:
#sub_test = test[(test.store_nbr == 25)].merge(items,on='item_nbr',how='left')

In [20]:
# test = test.merge(items,on='item_nbr',how='left')

In [21]:
# for cat in cats:
#     try:
#         sub1 = test[(test.family == cat) & (test.unit_sales > 0)][['date','unit_sales']]
#         sub1 = sub1.groupby('date')[['unit_sales']].mean()
#         sub2 = pred[cat]
#         sub2 = sub2[sub2.ds >= '2017-08-16'][['ds','yhat']]
#         sub1.merge(sub2,right_on='ds',left_index=True)[['unit_sales','yhat']].plot(title='{}'.format(cat))
#     except:
#         print sub1
#         print sub2
#         print "{}".format(cat)

In [22]:
# import pickle

# output = open('data/generated/prophet_test_forecast.pkl', 'wb')
# pickle.dump(pred, output)
# output.close()

# Run Prophet model for each month / category combo

In [23]:
# TO-DO!
# for each category create a Prophet model
## run for test set - DONE
## run for train set - RUNNING

# add rolling means info
## run for test set
## run for train set

# merge together rolling means + prophet

# cross validate performance
## run for last year in 2 week intervals

In [24]:
#sub_train = train_set[train_set.ds >= '2016-08-16']
train_set['ds'] = pd.to_datetime(train_set['ds'])

In [34]:
months = pd.date_range('2016-12-01','2017-09-01',freq='M')

In [35]:
import pickle

In [36]:
#pred_train = {month:{cat: pd.DataFrame() for cat in cats} for month in months}

for month in months:
    pred_train = {cat: pd.DataFrame() for cat in cats}
    
    prev_subset = train_set[train_set['ds'] <= month]
    report_month = month + pd.Timedelta('1D')
    
    for cat in cats:
        df = prev_subset[prev_subset.family == cat]
        df = df[['ds','y']].groupby('ds').mean()
        df['ds'] = df.index
        # Fit the category with Prophet
        try:
            m = Prophet()
            m.fit(df);
            future = m.make_future_dataframe(periods=report_month.days_in_month,freq='D')
            mon_pred = m.predict(future)
            pred_train[cat] = mon_pred[mon_pred.ds >= report_month]
        except:
            print "{}".format(month.date()) + ' ' + cat + ": no data points"
            
    output = open('data/generated/prophet_train_{}_forecast.pkl'.format(month.month), 'wb')
    pickle.dump(pred_train, output)
    output.close()